Um mit einem Datum-Index in einem pandas-DataFrame effektiv zu arbeiten, kannst du folgende Ansätze verwenden:

1. Den Index als Datum setzen

Falls dein Datum derzeit eine Spalte ist, kannst du diese als Index festlegen:

df = df.set_index('Date')

Dadurch wird die Spalte Date zum Index, und pandas interpretiert diesen als Datumsindex, was bestimmte zeitbezogene Operationen ermöglicht.

2. Sicherstellen, dass der Index als Datumsobjekt vorliegt

Falls der Index noch nicht als datetime-Objekt formatiert ist, konvertiere ihn:

df.index = pd.to_datetime(df.index)

Das ist besonders nützlich, wenn du Datumsoperationen wie Resampling oder Zeitreihenanalysen durchführen möchtest.

3. Zeitspezifische Operationen

Mit einem Datumsindex kannst du viele zeitspezifische Operationen durchführen:

a. Daten filtern

Du kannst Daten nach bestimmten Zeiträumen auswählen:

# Daten eines bestimmten Jahres
df_2023 = df['2023']

# Daten zwischen zwei Daten
df_range = df['2023-01-01':'2023-12-31']

b. Resampling

Das Resampling ist eine häufige Methode, um Daten auf bestimmte Zeiträume zu aggregieren, z.B. monatlich oder jährlich:

# Monatliche Mittelwerte
df_monthly = df.resample('M').mean()

# Jährliche Summen
df_yearly = df.resample('Y').sum()

c. Rolling-Aggregationen

Mit einem Datumsindex sind Rolling-Operationen (gleitende Fenster) nützlich, um Durchschnittswerte oder andere Aggregationen über bestimmte Zeiträume zu berechnen:

# Gleitender 3-Monats-Durchschnitt
df['3M_avg'] = df['Dividends'].rolling(window='90D').mean()

4. Plotten mit Datumsindex

Mit einem Datumsindex auf der X-Achse erstellt pandas automatisch geeignete Beschriftungen, was hilfreich ist, um Zeitreihendaten darzustellen:

df.plot(y='Dividends', title='Dividends over Time')

5. Zeitliche Differenzen berechnen

Wenn du Differenzen zwischen aufeinanderfolgenden Zeitpunkten berechnen möchtest, ist der Datumsindex hilfreich:

# Tägliche Differenz
df['daily_diff'] = df['Dividends'].diff()

Mit diesen Methoden kannst du zeitreihenbasierte Analysen und Manipulationen effizient mit einem Datumsindex durchführen.

In [5]:
import pandas as pd, os
testfile = "/Users/a2/code/fin/trade/data/financial/AT/daily/OPAP.AT.parquet"
df = pd.read_parquet(testfile)

In [6]:
df.head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2001-04-25 00:00:00+03:00,0.658620,0.745964,0.651538,0.665702,3571300,0.0,0.0
2001-04-26 00:00:00+03:00,0.672784,0.682226,0.625571,0.637374,720680,0.0,0.0
2001-04-27 00:00:00+03:00,0.637374,0.663341,0.627932,0.635013,647630,0.0,0.0
2001-04-30 00:00:00+03:00,0.637374,0.646817,0.616128,0.620850,225030,0.0,0.0
2001-05-01 00:00:00+03:00,0.620850,0.620850,0.620850,0.620850,0,0.0,0.0


In [11]:
import pandas as pd

def calculate_annual_dividend_yield(df):
    # Sicherstellen, dass der Index ein Datumsindex ist
    df.index = pd.to_datetime(df.index)
    
    # Gruppieren nach Jahr und Berechnung des jährlichen Dividend Yields
    annual_yield = df.resample('YE').apply({
        'Dividends': 'sum',  # Summe der Dividenden pro Jahr
        'Close': 'mean'      # Durchschnittlicher Schlusskurs pro Jahr
    })
    
    # Dividend Yield berechnen
    annual_yield['Dividend Yield'] = (annual_yield['Dividends'] / annual_yield['Close']) * 100
    
    # Relevante Spalten auswählen und Ergebnis zurückgeben
    return annual_yield[['Dividend Yield']]

# Anwenden der Funktion auf deinen DataFrame
annual_dividend_yield = calculate_annual_dividend_yield(df)
print(annual_dividend_yield)

                           Dividend Yield
Date                                     
2001-12-31 00:00:00+02:00       27.180319
2002-12-31 00:00:00+02:00       37.176159
2003-12-31 00:00:00+02:00       73.966466
2004-12-31 00:00:00+02:00       43.350583
2005-12-31 00:00:00+02:00       38.644030
2006-12-31 00:00:00+02:00       31.661807
2007-12-31 00:00:00+02:00       34.469601
2008-12-31 00:00:00+02:00       46.915629
2009-12-31 00:00:00+02:00       53.581483
2010-12-31 00:00:00+02:00       52.229029
2011-12-31 00:00:00+02:00       34.187183
2012-12-31 00:00:00+02:00       44.581241
2013-12-31 00:00:00+02:00       22.558453
2014-12-31 00:00:00+02:00        6.394229
2015-12-31 00:00:00+02:00       29.727759
2016-12-31 00:00:00+02:00       29.844715
2017-12-31 00:00:00+02:00       31.085145
2018-12-31 00:00:00+02:00        7.918847
2019-12-31 00:00:00+02:00       10.966609
2020-12-31 00:00:00+02:00       22.505385
2021-12-31 00:00:00+02:00        7.751116
2022-12-31 00:00:00+02:00       17

# Hier die ganze Berechnung.
### 1. dividend yield per year
### 2. dividend yield mean last 5 years
### 3. dividend mean per year last 3 years
 

In [2]:
# get data
import os, pandas as pd, json, re
p = "/Users/a2/code/fin/trade/data/financial/AT/daily"
with open("/Users/a2/code/fin/trade/static/tickers/AT_shitticker.json", "r") as f:
    sh = json.load(f)
files = os.listdir(p)
dfs = []
for f in files:
    ticker = re.match(r"(.*)\.parquet", f).group(1)
    if ticker in sh:
        continue
    df = pd.read_parquet(os.path.join(p, f), engine='pyarrow')
    dfs.append(df)

In [12]:
# 1
dividend_yields_per_stock = [calculate_annual_dividend_yield(x) for x in dfs] 

In [13]:
# 2 
last_5_y_yield = [x.tail(5)['Dividend Yield'].mean() for x in dividend_yields_per_stock]

In [16]:
# 3
def average_days_between_dividends(df):
    # Sicherstellen, dass der Index ein Datumsindex ist
    df.index = pd.to_datetime(df.index)
    
    # Aktuelles Datum und Berechnung des Startdatums (5 Jahre zurück)
    end_date = df.index.max()
    start_date = end_date - pd.DateOffset(years=5)
    
    # Filtern der Daten auf die letzten 5 Jahre und nur positive Dividendenwerte
    recent_dividends = df[(df.index >= start_date) & (df['Dividends'] > 0)]
    
    # Prüfen, ob genügend Dividenden vorhanden sind
    if recent_dividends.shape[0] < 2:
        return None  # Keine oder nur eine Dividende vorhanden, keine Berechnung möglich
    
    # Nur die letzten 10 Dividenden auswählen, falls mehr als 10 vorhanden sind
    recent_dividends = recent_dividends.tail(10)
    
    # Berechnen der Tage zwischen den Dividendenzahlungen
    days_between_dividends = recent_dividends.index.to_series().diff().dropna().dt.days
    
    # Durchschnitt der Tage zwischen Dividendenzahlungen berechnen
    average_days = days_between_dividends.mean()
    
    return average_days


avg_div_payment_days = [average_days_between_dividends(x) for x in dfs]